## Connecting to AWS BedRock

**References**

1. [Integrating Foundation Model with Amazon Bedrock](https://aws.amazon.com/blogs/machine-learning/integrate-foundation-models-into-your-code-with-amazon-bedrock/)

In [ ]:
import configparser
import json
import boto3
import logging
from botocore.exceptions import ClientError

### Initiate Services

In [ ]:
logging.basicConfig(
    format="%(levelname)s - %(asctime)s - %(message)s", level=logging.DEBUG
)

logger = logging

### Reading Config File

In [ ]:
CONFIG_FILENAME=".config.ini"

config = configparser.ConfigParser()
config.read(CONFIG_FILENAME)

def get_value_by_section_and_key(section, key):
        """get_value_by_section_and_key"""
        return config.get(section, key)

def get_all_details_of_section(section) -> dict:
    """get_all_details_of_section"""
    return dict(config.items(section))


default_cfgs = get_all_details_of_section("DEFAULT")

### Create boto3 client - BEDROCK

In [ ]:
bedrock_client = boto3.client(
    'bedrock',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)

def list_foundation_models():
    """
    List the available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models
    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise

all_models = list_foundation_models()
# all_models
# len(all_models) # 73

### Calling a Model

In [ ]:
kwargs = {
 "modelId": "amazon.titan-text-express-v1",
 "contentType": "application/json",
 "accept": "application/json",
 "body": "{\"inputText\":\"Hello, who are you?\",\"textGenerationConfig\":{\"maxTokenCount\":8192,\"stopSequences\":[],\"temperature\":0,\"topP\":1}}"
}
# print(kwargs)

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())
response_body